In [27]:
from sklearn import datasets, svm
import numpy as np

## Score and $k$-fold crossvalidation

First let's calculate the score as before!

In [26]:
svc = svm.SVC(C=1, kernel='linear')
svc.fit(X=digits.data[:-100], y=digits.target[:-100])
svc.score(X=digits.data[-100:], y=digits.target[-100:])

0.98

Now repeat this procedure $k-1$ times

In [67]:
X_folds = np.array_split(digits.data, 3)
y_folds = np.array_split(digits.target, 3)
def split_2_1(k, data):
    train = data.copy()
    test = train.pop(k)
    train = np.concatenate(train)
    res = {'train': train, 'test': test}
    return(res)
def score_2_1(k, X_folds=X_folds, y_folds=y_folds, cls=svm.SVC(C=1, kernel='linear')):
    X = split_2_1(k, X_folds)
    y = split_2_1(k, y_folds)
    cls.fit(X=X['train'], y=y['train'])
    score = cls.score(X=X['test'], y=y['test'])
    return(score)
[score_2_1(k) for k in range(3)]

[0.9348914858096828, 0.9565943238731218, 0.9398998330550918]

In [2]:
%connect_info

{
  "shell_port": 39953,
  "iopub_port": 56643,
  "stdin_port": 44071,
  "control_port": 37353,
  "hb_port": 46839,
  "ip": "127.0.0.1",
  "key": "0d67b865-174cbf4754a1f3315b6cecc7",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-df67f196-8fc4-4abd-ba8e-e71d9b19272d.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
